# Select GPU if needed

In [1]:
import os
gpu_number = 0
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_number)

# Import modules

In [2]:
import torch
import torchvision.transforms as transforms

from tools.metrics import get_metrics
from tools.dataset import MaskRCNNDataset
from model import ssl_maskrcnn

# Set parameters

In [3]:
weights_file = 'outputs/dinov2_s_final_weights.pt'
dino_model = 'dinov2_s' # Must be configured according to the model of the weights_file
num_classes = 2

# Configure model

In [4]:
model = ssl_maskrcnn(num_classes, dino_model)
weights = torch.load(weights_file, weights_only=True)
model.load_state_dict(weights)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Using cache found in /home/wormhole/.cache/torch/hub/facebookresearch_dinov2_main
/home/wormhole/.conda/envs/dfuc/lib/python3.8/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/wormhole/.conda/envs/dfuc/lib/python3.8/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
/home/wormhole/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/home/wormhole/.cache/torch/hub/facebookr

ssl_maskrcnn(
  (backbone): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-11): 12 x NestedTensorBlock(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )

# Set valid parameters

In [5]:
val_image_dir = 'data/valid/image'
val_mask_dir  = 'data/valid/mask'

# Set valid dataset

In [6]:
img_transform = transforms.Compose([
    transforms.Resize((14*64,14*64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

mask_transform = transforms.Compose([
    transforms.Resize((32*32,32*32)),
    transforms.ToTensor(),
])

valid_dataset = MaskRCNNDataset(img_dir=val_image_dir, mask_dir=val_mask_dir, num_classes=num_classes, img_transform=img_transform, mask_transform=mask_transform)

# Get metrics: dsc, iou, fne, fpe

In [7]:
get_metrics(model, valid_dataset)

mask_threshold=0.5, score_threshold=0.2
Loaded data number: 400


Get dice progress:   0%|          | 0/400 [00:00<?, ?it/s]

{'dsc': 0.6702604640943445,
 'iou': 0.5697198864868288,
 'fne': 0.25343452142423006,
 'fpe': 0.005667870053076916}